In [17]:
import pickle
import numpy as np
import pandas as pd
import polars as pl
import scanpy as sc

In [18]:
df_real = pl.read_parquet("/home/wzc26/work/vcc/nbglm/data/all_de_cache/real_de.parquet")

In [19]:
df = pd.read_csv("/home/wzc26/work/vcc/nbglm/outputs/poisson_reg_test__20251009_142752/metrics/de.csv")
df_wrong = df[df['gt_significant']!=df['pred_significant']]

In [20]:
df_wrong

,gene,pert,gt_pvals,gt_fdr,gt_logfoldchanges,prediction_pvals,prediction_fdr,prediction_logfoldchanges,gt_significant,pred_significant
1,NOC2L,MTA1,0.330293,0.686575,0.030344,9.272703e-08,4.051747e-07,0.146240,False,True
5,HES4,MTA1,0.969067,1.000000,0.052484,5.806111e-03,1.461993e-02,0.938944,False,True
6,ISG15,MTA1,0.807070,0.960492,-0.014712,9.106220e-03,2.210597e-02,0.066071,False,True
7,AGRN,MTA1,0.131783,0.412016,0.011119,7.114760e-10,3.631018e-09,-0.058592,False,True
18,ACAP3,MTA1,0.118644,0.386109,-0.049633,8.164828e-04,2.352474e-03,0.082777,False,True
...,...,...,...,...,...,...,...,...,...,...
542385,NLGN4Y,SALL4,0.147993,0.442046,-0.111007,5.603695e-05,1.876339e-04,0.268803,False,True
542386,KDM5D,SALL4,0.337016,0.693490,0.047222,2.326162e-09,1.145656e-08,0.304874,False,True
542394,MT-ND3,SALL4,0.312599,0.668467,0.065208,4.709666e-17,3.549871e-16,0.333829,False,True
542398,MT-ND6,SALL4,0.009687,0.062799,0.269046,5.541991e-19,4.558307e-18,0.590069,False,True


In [21]:
import polars as pl

alpha = 0.05  # 显著性阈值

genes_never_sig = (
    df_real.lazy()
      .group_by("feature")
      .agg([
          (pl.col("fdr") < alpha).any().alias("any_sig"),       # 是否在任一target下显著
          pl.n_unique("target").alias("n_targets"),              # 该基因被测试的target数
          pl.min("fdr").alias("min_fdr")                         # 观察到的最小FDR（越大越“不变”）
      ])
      .filter(pl.col("any_sig") == False)                        # 从未显著
      .select(["feature", "n_targets", "min_fdr"])
      .collect()
)


In [22]:
genes_never_sig

feature,n_targets,min_fdr
cat,u32,f32
"""LRP5L""",150,0.127757
"""OR1L4""",150,1.0
"""JCHAIN""",150,0.05086
"""PCK1""",150,0.070052
"""OR4S1""",150,0.104456
…,…,…
"""OR5H15""",150,0.933847
"""WEE2""",150,0.074082
"""GPIHBP1""",150,0.067842


In [23]:
wrong_gene=[]
for gene in genes_never_sig['feature']:
    if gene in df_wrong['gene'].values:
        wrong_gene.append(gene)
len(wrong_gene)

104

In [24]:
import pandas as pd
df = pd.read_csv("/home/wzc26/work/vcc/nbglm/outputs/poisson_reg_test__20251009_142752/metrics/de.csv")
df = pd.read_csv("/home/wzc26/work/vcc/nbglm/outputs/gpm_default__20251009_150340/metrics/de.csv")
S = ((df.gt_significant) & (df.pred_significant)).sum()
V = ((~df.gt_significant) & (df.pred_significant)).sum()
T = ((df.gt_significant) & (~df.pred_significant)).sum()
U = ((~df.gt_significant) & (~df.pred_significant)).sum()

m  = S+T+U+V
R  = S+V
ppv = S / R if R else 0.0
tpr = S / (S+T) if (S+T) else 0.0
fdr = V / R if R else 0.0


In [1]:
import pandas as pd
import numpy as np
import pandas as pd
df = pd.read_csv("/home/wzc26/work/vcc/nbglm/outputs/poisson_reg_test__20251009_142752/metrics/de.csv")
df = pd.read_csv("/home/wzc26/work/vcc/nbglm/outputs/gpm_default__20251009_150340/metrics/de.csv")
S = ((df.gt_significant) & (df.pred_significant)).sum()
V = ((~df.gt_significant) & (df.pred_significant)).sum()
T = ((df.gt_significant) & (~df.pred_significant)).sum()
U = ((~df.gt_significant) & (~df.pred_significant)).sum()

m  = S+T+U+V
R  = S+V
ppv = S / R if R else 0.0
tpr = S / (S+T) if (S+T) else 0.0
fdr = V / R if R else 0.0
# ---- 1) 四格表（Confusion table）与总体指标 ----
# 行：Pred (是否判为显著)；列：GT (真实是否显著/差异)
ct = pd.crosstab(
    df['pred_significant'], df['gt_significant'],
    rownames=['Pred (reject H0)'], colnames=['GT (HA true)']
).reindex(index=[True, False], columns=[True, False], fill_value=0)

# 统一命名方便阅读
ct.index = ['Pred: significant', 'Pred: non-significant']
ct.columns = ['GT: significant', 'GT: non-significant']

# 取四个格子
S = int(ct.loc['Pred: significant', 'GT: significant'])      # True Positive
V = int(ct.loc['Pred: significant', 'GT: non-significant'])  # False Positive (Type I)
T = int(ct.loc['Pred: non-significant', 'GT: significant'])  # False Negative (Type II)
U = int(ct.loc['Pred: non-significant', 'GT: non-significant'])  # True Negative

m  = S + V + T + U
R  = S + V          # 被拒绝的原假设 (rejections)
m0 = U + V          # 真实原假设数 (true nulls)

def sdiv(a, b):
    return float(a) / float(b) if b else 0.0

overall_metrics = pd.Series({
    'TP (S)': S,
    'FP (V)': V,
    'FN (T)': T,
    'TN (U)': U,
    'm (total)': m,
    'R (rejections)': R,
    'm0 (true nulls)': m0,
    'Precision / PPV': sdiv(S, S+V),
    'Recall / TPR / Power': sdiv(S, S+T),
    'FDR (FDP)': sdiv(V, R),           # 约定 R=0 时取 0
    'FNR': sdiv(T, S+T),
    'Accuracy': sdiv(S+U, m),
    'F1': sdiv(2*(sdiv(S, S+V)*sdiv(S, S+T)), (sdiv(S, S+V)+sdiv(S, S+T))) if (S+V) and (S+T) else 0.0
})

display(ct)
display(overall_metrics.to_frame('value'))

# ---- 2) 按 pert 分组统计与指标 ----
def counts_metrics(g: pd.DataFrame) -> pd.Series:
    ctab = pd.crosstab(g['pred_significant'], g['gt_significant'])
    # 防止缺列缺行
    s = int(ctab.get(True, {}).get(True, 0))
    v = int(ctab.get(True, {}).get(False, 0))
    t = int(ctab.get(False, {}).get(True, 0))
    u = int(ctab.get(False, {}).get(False, 0))
    m  = s+v+t+u
    r  = s+v
    return pd.Series({
        'TP (S)': s, 'FP (V)': v, 'FN (T)': t, 'TN (U)': u,
        'm': m, 'R': r, 'm0': u+v,
        'PPV': sdiv(s, r),
        'TPR': sdiv(s, s+t),
        'FDR': sdiv(v, r),
        'FNR': sdiv(t, s+t),
        'Accuracy': sdiv(s+u, m),
        'F1': sdiv(2*(sdiv(s, r)*sdiv(s, s+t)), (sdiv(s, r)+sdiv(s, s+t))) if r and (s+t) else 0.0
    })

by_pert = df.groupby('pert', sort=False).apply(counts_metrics)
# 可按需要排序，比如按 FDR 升序或 TPR 降序
by_pert_sorted = by_pert.sort_values(['FDR','TPR'], ascending=[True, False])

display(by_pert_sorted)


,GT: significant,GT: non-significant
Pred: significant,41560,185630
Pred: non-significant,37468,277742


,value
TP (S),41560.000000
FP (V),185630.000000
FN (T),37468.000000
TN (U),277742.000000
m (total),542400.000000
R (rejections),227190.000000
m0 (true nulls),463372.000000
Precision / PPV,0.182931
Recall / TPR / Power,0.525890
FDR (FDP),0.817069


/mnt/nvme0n1/users/wzc26/.tmp/ipykernel_2963900/3721399715.py:80: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  by_pert = df.groupby('pert', sort=False).apply(counts_metrics)


,TP (S),FP (V),FN (T),TN (U),m,R,m0,PPV,TPR,FDR,FNR,Accuracy,F1
pert,,,,,,,,,,,,,
CASP2,544.0,116.0,11815.0,5605.0,18080.0,660.0,5721.0,0.824242,0.044017,0.175758,0.955983,0.340100,0.083570
FOXH1,1887.0,411.0,9805.0,5977.0,18080.0,2298.0,6388.0,0.821149,0.161392,0.178851,0.838608,0.434956,0.269764
UQCRQ,2757.0,858.0,7613.0,6852.0,18080.0,3615.0,7710.0,0.762656,0.265863,0.237344,0.734137,0.531471,0.394280
POLB,790.0,288.0,9470.0,7532.0,18080.0,1078.0,7820.0,0.732839,0.076998,0.267161,0.923002,0.460288,0.139354
CLDN7,1184.0,464.0,7933.0,8499.0,18080.0,1648.0,8963.0,0.718447,0.129867,0.281553,0.870133,0.535564,0.219972
TFAM,3208.0,1451.0,5104.0,8317.0,18080.0,4659.0,9768.0,0.688560,0.385948,0.311440,0.614052,0.637445,0.494642
SSBP1,2128.0,963.0,5441.0,9548.0,18080.0,3091.0,10511.0,0.688450,0.281147,0.311550,0.718853,0.645796,0.399250
ACVR1B,1045.0,508.0,7361.0,9166.0,18080.0,1553.0,9674.0,0.672891,0.124316,0.327109,0.875684,0.564768,0.209860
MTA1,1838.0,894.0,6185.0,9163.0,18080.0,2732.0,10057.0,0.672767,0.229091,0.327233,0.770909,0.608462,0.341795
